In [2]:
import matplotlib.pyplot as plt
import cv2, re
import numpy as np
from queue import PriorityQueue

In [48]:
# BMP to JPEG conversion
img = cv2.imread('snail.bmp')

s = img.shape
new_shape = (((s[0] + 15) // 16) * 16, ((s[1] + 15) // 16) * 16, s[2])
new_image = np.zeros(new_shape, dtype = 'float32')
new_image[:s[0], :s[1], :] = img
img = np.array(new_image)
old_shape = s
s = new_shape
s1 = (s[0] // 2, s[1] // 2)

# todo - padding

# conversion from RGB format to YCbCr (Y -> Luminance, Cb -> blue chrominance, Cr -> red chrominance)
imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
Y = imgYCC[:,:,0]
Cb = np.zeros(s1, dtype = int)
Cr = np.zeros(s1, dtype = int)

# average every 2x2 block for Cb and Cr
for i in range(s[0]//2):
    for j in range(s[1]//2):
        x = i*2
        y = j*2
        avg = (img[x, y, 1] + img[x, y + 1, 1] + img[x + 1, y, 1] + img[x + 1, y + 1, 1]) // 4
        Cb[i, j] = avg
        avg = (img[x][y][2] + img[x][y + 1][2] + img[x + 1][y][2] + img[x + 1][y + 1][2]) // 4

# Discrete Cosine Transform
Y -= 128
Cb -= 128
Cr -= 128
Y_blocks = []
Cb_blocks = []
Cr_blocks = []
for i in range(0, s[0], 8):
    for j in range(0, s[1], 8):
        grid = np.float32(Y[i:i+8, j:j+8])
        Y_blocks.append(cv2.dct(grid))
for i in range(0, s1[0], 8):
    for j in range(0, s1[1], 8):
        grid = np.float32(Cb[i:i+8, j:j+8])
        Cb_blocks.append(cv2.dct(grid))
        grid = np.float32(Cr[i:i+8, j:j+8])
        Cr_blocks.append(cv2.dct(grid))

len_y = len(Y_blocks)
len_c = len(Cb_blocks)

# Quantization
Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
C_q = np.array([[4., 4, 6, 10, 21, 21, 21, 21], [4, 5, 6, 21, 21, 21, 21, 21], [6, 6, 12, 21, 21, 21, 21, 21], [10, 14, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21]])
for i in range(len_y):
    Y_blocks[i] //= Y_q
for i in range(len_c):
    Cb_blocks[i] //= C_q
for i in range(len_c):
    Cr_blocks[i] //= C_q

# Run length encoding
def spiral_traversal(block):
    p = 8
    arr = np.array([])
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr = np.append(arr, block[i][k-i])
            else :
                arr = np.append(arr, block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr = np.append(arr, block[i][j])
            else:
                arr = np.append(arr, block[j][i])
        k+=1
    return arr

dc = 0
def encode(block):
    global dc
    temp = spiral_traversal(block)
    l = len(temp)

    encoded_block = []
    encoded_block.append((0, temp[0] - dc))
    dc = temp[0]
    c0 = 0
    for i in range(1, l):
        if temp[i] or c0==15 :
            encoded_block.append((c0, temp[i]))
            c0 = 0
        else:
            c0 += 1
    encoded_block.append((0, 0))

    encoded_block = np.array(encoded_block, dtype = int)
    return encoded_block

encoded_Y = list()
encoded_Cr = list()
encoded_Cb = list()
for i in range(len_y):
    encoded_Y.append(encode(Y_blocks[i]))
for i in range(len_c):
    encoded_Cb.append(encode(Cb_blocks[i]))
    encoded_Cr.append(encode(Cr_blocks[i]))
encoded_Y = np.array(encoded_Y, dtype = object)
encoded_Cr = np.array(encoded_Cr, dtype = object)
encoded_Cb = np.array(encoded_Cb, dtype = object)

# print(encoded_Y, encoded_Cb, encoded_Cr, sep = '\n')

# Huffman Tables
def get_cat(num): ##extracting position of msb to determine what length of bits it will need to be encoded
    num = int(abs(num))
    ans = 0
    pwr = 1
    while pwr < num:
        pwr <<= 1
        ans += 1
    return ans
##***********************************************************DANGER**********************


def make_freq_table(dc_freq, ac_freq, encoded_blocks):
    for block in encoded_blocks:
        #block of length 64 usually (we take 8*8 grids), first element is dc coeff, the rest are ac coeffs
        #dc block[0] is encoded as (0, value)
        dc = block[0]
        dc_run_length, dc_val = dc #dc_run_length will be 0
        
        cat = get_cat(dc_val)
        dc_freq[dc_run_length, cat] += 1
        
        for i in range(1, len(block)):
            
            #ac block[i] is encoded as (run length of zeros before value max 15, value of ith non zero ac coefficient)
            ac_i = block[i]
            run_length, value = ac_i
            cat = get_cat(value) # find the category of value
            ac_freq[run_length, cat] += 1
            
dc_huffman_Y = np.zeros((1, 16), dtype=int)
ac_huffman_Y = np.zeros((16,16), dtype=int)
dc_huffman_C = np.zeros((1, 16), dtype=int)
ac_huffman_C = np.zeros((16,16), dtype=int)

make_freq_table(dc_huffman_Y, ac_huffman_Y, encoded_Y)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cr)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cb)



# make huffman codes
depth = 0
def get_codebook(freq):
    codebook = np.empty(freq.shape, dtype = object)
    def get_code_lengths(freq):

        def make_tree(freq):
            global code
            q = PriorityQueue()
            m = len(freq)
            n = len(freq[0])

            cnt = 0
            for i in range(m):
                for j in range(n):
                    if freq[i,j]:
                        q.put((freq[i,j], str((i,j))))
                        cnt += 1

            for i in range(cnt-1):
                a = q.get()
                b = q.get()
                q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

            tree = eval(q.get()[1])
            return tree

        tree = make_tree(freq)
        code_lengths = np.zeros(freq.shape, dtype=int)

        def get_depths(arr, code_lengths):
            global depth
            if type(arr[1]) == int and type(arr[0]) == int:
                code_lengths[arr[0], arr[1]] = depth
            else:
                depth += 1
                get_depths(arr[0], code_lengths)
                get_depths(arr[1], code_lengths)
                depth-=1

        get_depths(tree, code_lengths)
        return code_lengths
    
    code_lengths = get_code_lengths(freq)
    length_symbol_pairs = []
    
    m, n = freq.shape
    for i in range(m):
        for j in range(n):
            if code_lengths[i, j]:
                length_symbol_pairs.append((code_lengths[i, j], (i, j)))
    length_symbol_pairs.sort()
    
    def get_code(code, length):
        st = bin(code).replace("0b", "")[::-1]
        while len(st) < length:
            st += '0'
        return st[::-1]
    
    code = 0
    cur_len = 0
    for length, symbol in length_symbol_pairs:
        while cur_len < length:  #if there is no symbol with the current length
            cur_len += 1
            code *= 2
        codebook[symbol[0], symbol[1]] = get_code(code, length)
        code += 1
    
    return codebook, length_symbol_pairs

codes_dc_Y, sorted_symbols_dc_Y = get_codebook(dc_huffman_Y)
codes_ac_Y, sorted_symbols_ac_Y = get_codebook(ac_huffman_Y)
codes_dc_C, sorted_symbols_dc_C = get_codebook(dc_huffman_C)
codes_ac_C, sorted_symbols_ac_C = get_codebook(ac_huffman_C)

def to_two_bytes(num):
    bh = bin(num).replace("0b", "")
    bh = bh[::-1]
    while len(bh) < 16:
        bh += '0'
    bh = bh[::-1]

    return [int(bh[:8], 2), int(bh[8:], 2)]

# Writing jpeg image in binary
jpeg_image = [0xff, 0xd8, 0xff, 0xe0, 0x00, 0x10]

header = [0x4a, 0x46, 0x49, 0x46, 0x00, 0x01, 0x01, 0x01, 0x00, 0x48, 0x00, 0x48, 0x00, 0x00, 0xff]
jpeg_image.extend(header)

luminance_quantisation_table = [0xff, 0xdb, 0x00, 0x43]
for ele in Y_q:
    for x in ele:
        luminance_quantisation_table.append(int(x))
jpeg_image.extend(luminance_quantisation_table)

chrominance_quantisation_table = [0xff, 0xdb, 0x00, 0x43]
for ele in C_q:
    for x in ele:
        chrominance_quantisation_table.append(int(x))
jpeg_image.extend(chrominance_quantisation_table)

start_of_frame = [0xff, 0xc0, 0x00, 0x11, 0x08]
start_of_frame.extend(to_two_bytes(old_shape[0]))
start_of_frame.extend(to_two_bytes(old_shape[1]))
start_of_frame.extend([0x03, 0x01, 0x22, 0x00, 0x02, 0x11, 0x01, 0x03, 0x11, 0x01])
jpeg_image.extend(start_of_frame)

def parse_table(sorted_symbols, typ):
    table = [0xff, 0xc4]
    ln = 19
    symbols = []
    freqs = [0 for i in range(16)]
    
    for length, symbol in sorted_symbols:
        freqs[length] += 1
        symbols.append(symbol[0] * 16 + symbol[1])
        ln += 1
    
    table.extend(to_two_bytes(ln))
    table.append(typ)
    table.extend(freqs)
    table.extend(symbols)
    return table

jpeg_image.extend(parse_table(sorted_symbols_dc_Y, 0x00))
jpeg_image.extend(parse_table(sorted_symbols_ac_Y, 0x10))
jpeg_image.extend(parse_table(sorted_symbols_dc_C, 0x01))
jpeg_image.extend(parse_table(sorted_symbols_ac_C, 0x11))

start_of_scan = [0xff, 0xda, 0x00, 0x0c, 0x03, 0x01, 0x00, 0x02, 0x11, 0x03, 0x11, 0x00, 0x3f, 0x00]
jpeg_image.extend(start_of_scan)

def bit_rep(value):
    if value == 0:
        return ''
    else value>0 : 
        v = bin(abs(value)).replace("0b", '')
        if value>0 :
            return v
        else:
            return ~v

def encode_data(data_blocks, dc_codebook, ac_codebook):
    for block in data_blocks:
        dc_val = block[0][1]
        
        
        
data = []

end_of_image = [0xff, 0xd9]
jpeg_image.extend(end_of_image)

try:
    with open("my_image.jpeg", 'wb') as f:
        for byte in jpeg_image:
            f.write(byte.to_bytes(1, byteorder='big'))
except Exception as e:
    print(e)

file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()

l = re.findall('..?', s)
d = np.reshape(l, (-1, 1))
d = [' '.join(ele) for ele in d]
for ele in d:
    print(ele)
file.close()

ff
d8
ff
e0
00
10
4a
46
49
46
00
01
01
01
00
48
00
48
00
00
ff
ff
db
00
43
04
03
04
04
04
06
0b
0f
03
03
03
04
05
08
0e
13
03
04
04
05
08
0c
10
14
04
05
06
07
0c
0e
12
14
06
06
09
0b
0e
11
15
17
09
0c
0c
12
17
16
19
15
0b
0d
0f
11
15
17
19
15
0d
0c
0c
0d
10
13
15
15
ff
db
00
43
04
04
06
0a
15
15
15
15
04
05
06
15
15
15
15
15
06
06
0c
15
15
15
15
15
0a
0e
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
ff
c0
00
11
08
01
00
01
00
03
01
22
00
02
11
01
03
11
01
ff
c4
00
1d
00
00
01
00
02
03
01
01
02
00
00
00
00
00
00
00
00
00
05
06
03
04
07
08
02
01
09
ff
c4
00
40
01
00
00
01
04
02
01
03
03
03
03
01
06
02
08
08
00
00
01
02
03
f0
04
10
05
06
11
20
07
12
30
13
21
40
08
14
22
50
15
23
31
32
60
70
16
41
17
24
25
26
33
42
71
80
18
34
35
51
52
61
a0
c0
ff
c4
00
16
10
00
01
02
00
00
00
00
00
00
00
00
00
00
00
00
00
09
00
08
ff
c4
00
35
11
00
00
02
01
03
03
04
00
05
04
00
04
08
00
00
00
00
f0
01
02
03
10
04
05
20
06
11
12
30
07
13
2

In [3]:
# encoded_Y
# (int)(np.log2(abs(-21))+1)

# dc_huffman = np.random.randint(160, size=16)
dc_huffman = [ 25,2, 90,  11, 108 , 36,  23, 107, 145,   0,  40,  76,  71, 146, 151  21]
print(dc_huffman)
q = PriorityQueue()

for i in range(16):
    q.put((dc_huffman[i], (i,)))

for i in range(15):
    a = q.get()
    b = q.get()
    q.put((a[0]+b[0], (a[1], b[1])))
    
f = q.get()
print(f)

codes_dc_Y = ['' for i in range(16)]
code = ""
def create_codes(arr):
    global code
    if len(arr) == 1:
        codes_dc_Y[arr[0]] = code
    else:
        code += "0"
        create_codes(arr[0])
        code = code[:-1]
        code += "1"
        create_codes(arr[1])
        code = code[:-1]
create_codes(f[1])
print(codes_dc_Y)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (947485319.py, line 5)

In [4]:

dc_huffman_Y = np.array([ 25,2, 90,  11, 108 , 36,  23, 107, 145,   0,  40,  76,  71, 146, 151,  21]).reshape((16, 1))
ac_huffman_Y = np.array([[ 57,  16,  42, 157,  71,  67, 153,  90,   6,  51,  51,  38, 137,
         77, 156,  62],
       [ 94, 158, 116,  40,  21, 112, 109,  10,  69, 104, 115,  38,  56,
         17,  98, 136],
       [ 26,  68, 107, 131, 121, 132,  12, 100,  27,  91,  58,  14, 142,
         66,  11, 102],
       [ 88, 118,  58, 147, 110, 118, 125,  22,  13,  24,  96, 113,  16,
        111, 122, 110],
       [158, 118,  14,  84, 103, 155, 116,  43,  49, 133,  13,  13,  13,
          4, 148,   4],
       [  9, 145,  55,  43,  20,  24,  89,  44,  42,  93, 132,  60,  94,
         89,  61,  87],
       [ 19, 132, 125, 113,  29,  49,  67,  32, 104, 138,  83,  97, 130,
         36,  77,  15],
       [ 92, 108,  21,  81,  67, 138, 113,   9, 144,  46,  50,  87,  65,
        111, 127,  35],
       [152,  86,  59, 159,  32,  50,   4, 111, 150, 137,  51,   4,  80,
         50,   7, 158],
       [ 65,  47,  55,  61, 125, 130,  27, 106,  46,  17,  76, 109,  47,
         49, 146,  79],
       [  6, 157,  85,  51,  22, 128,  33,  59,  54,  61,  65,  82,  75,
         17,  87, 112],
       [142, 153,  46,  17,   1, 152,  58,  39,  52, 105, 111, 138,  61,
         85, 148,  15],
       [117, 128, 116,  67, 141, 132,  64,  65,  99,  48, 154,  86,  70,
         80,  82,  52],
       [ 92,  49,  17,  12,  22,  98,  31,  84,  52,   0,  59,  17,  52,
         69,  89, 113],
       [134,  65,  90,  46,  31, 122,  35, 154, 123,  12, 133,  39, 119,
         44,  60,  45],
       [ 16,  81, 149,  42,  17,  54, 146,  49,  76,   6,  37, 152,  83,
         27, 123,  17]]).reshape((16,16))
#[[4599 1687 1701 1464 1007  598  379  187   58    0    0    0    0    0
#      0    0]
#  [ 786  248  199   96   42    9    7    2    1    0    0    0    0    0
#      0    0]
#  [ 326   63   36   13    2    2    2    0    0    0    0    0    0    0
#      0    0]
#  [ 116   12    9    2    1    1    0    0    0    0    0    0    0    0
#      0    0]
#  [  60    6    3    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [  20    1    1    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [  12    1    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   8    2    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   3    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   1    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   1    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]
#  [2301    0    0    0    0    0    0    0    0    0    0    0    0    0
#      0    0]]
code = ""
def get_codebook(freq):
    q = PriorityQueue()
    m = len(freq)
    n = len(freq[0])
    codebook = np.empty((m, n), dtype = object)

    cnt = 0
    for i in range(m):
        for j in range(n):
            if freq[i,j]:
                q.put((freq[i,j], str((i,j))))
                cnt += 1

    for i in range(cnt - 1):
        a = q.get()
        b = q.get()
        q.put((a[0]+b[0], str((eval(a[1]), eval(b[1])))))

    tree = eval(q.get()[1])
    print(tree)
    
    def create_codes(arr, codebook):
        global code
        if type(arr[1]) == int and type(arr[0]) == int:
            codebook[arr[0], arr[1]] = code
        else:
            code += "0"
            create_codes(arr[0], codebook)
            code = code[:-1]
            code += "1"
            create_codes(arr[1], codebook)
            code = code[:-1]
            
    create_codes(tree, codebook)
    return codebook

codes_dc_Y = get_codebook(dc_huffman_Y)
print(codes_dc_Y)

codes_ac_Y = get_codebook(ac_huffman_Y)
print(codes_ac_Y)

((((2, 0), (7, 0)), ((4, 0), (((((1, 0), (3, 0)), (15, 0)), (5, 0)), (12, 0)))), (((8, 0), (13, 0)), ((14, 0), ((11, 0), ((10, 0), ((6, 0), (0, 0)))))))
[['111111']
 ['0110000']
 ['000']
 ['0110001']
 ['010']
 ['01101']
 ['111110']
 ['001']
 ['100']
 [None]
 ['11110']
 ['1110']
 ['0111']
 ['101']
 ['110']
 ['011001']]
((((((((3, 11), (6, 3)), ((7, 6), ((0, 0), (11, 6)))), (((1, 10), ((2, 10), (3, 2))), ((1, 2), (12, 2)))), ((((4, 6), (12, 0)), (((10, 7), (13, 10)), (3, 1))), (((3, 5), (4, 1)), (((8, 2), ((6, 4), ((6, 15), (0, 1)))), (14, 12))))), ((((((14, 14), (5, 11)), (2, 4)), (((10, 9), (11, 12)), ((5, 14), (9, 3)))), (((14, 5), (3, 14)), ((14, 8), (15, 14)))), ((((((13, 6), (14, 4)), (0, 15)), (3, 6)), ((6, 2), (9, 4))), (((7, 14), ((((15, 0), (3, 12)), (6, 7)), (12, 6))), ((10, 5), (12, 1)))))), ((((((((8, 4), (10, 6)), (10, 10)), ((12, 7), (14, 1))), (((7, 12), (9, 0)), (6, 12))), (((9, 5), (2, 3)), ((12, 5), (2, 5)))), ((((5, 10), (6, 1)), (((2, 13), (0, 5)), (14, 10))), (((4, 

In [34]:
# np.random.randint(160,size=(16,16))
# for ele in encoded_Y:
#     cat = get_msb_pos(ele[0][1])
#     dc_huffman_Y[0, cat] += 1

#     for i in range(1, len(ele)):
#         # find the category
#         cat = get_msb_pos(ele[i][1])
#         ac_huffman_Y[ele[i][0], cat] += 1

# for ele in encoded_Cr:
#     cat = get_msb_pos(ele[0][1])
#     dc_huffman_C[0, cat] += 1                           

#     for i in range(1, len(ele)):
#         # find the category
#         cat = get_msb_pos(ele[i][1])
#         ac_huffman_C[ele[i][0], cat] += 1

# for ele in encoded_Cb:
#     cat = get_msb_pos(ele[0][0])
#     dc_huffman_C[0, cat] += 1                           

#     for i in range(1, len(ele)):
#         # find the category
#         cat = get_msb_pos(ele[i][1])
#         ac_huffman_C[ele[i][0], cat] += 1

In [32]:
# def spiral_traversal(block):
#     p = 8
#     arr = np.array([])
#     k = 0
#     for c in range(p):
#         for i in range(0, k+1):
#             if k%2!=0 :
#                 arr = np.append(arr, block[i][k-i])
#             else :
#                 arr = np.append(arr, block[k-i][i])
#         k+=1
        
#     k = 1
#     for c in range(p-1):
#         j = p
#         for i in range(k, p):
#             j-=1
#             if k%2 == 0:
#                 arr = np.append(arr, block[i][j])
#             else:
#                 arr = np.append(arr, block[j][i])
#         k+=1
#     return arr
        
# Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
# print(Y_q)
# print(spiral_traversal(Y_q))

In [33]:
# arr = np.zeros([])
# arr = np.append(arr,1)
# arr

In [30]:
# img = np.asarray(cv2.imread('waterpool.bmp', 0))
# print(list(img[0]))

In [31]:
# img = cv2.imread('waterpool.jpeg')
# print(img)

In [36]:
file = open('snail.jpeg', 'rb')
data = file.read()
s = data.hex()

import re
l = re.findall('..?', s)
print(len(l))
d = np.reshape(l, (-1, 11))
d = [' '.join(ele) for ele in d]
for ele in d:
    print(ele)
file.close()

13211
ff d8 ff e0 00 10 4a 46 49 46 00
01 01 01 00 60 00 60 00 00 ff db
00 43 00 03 02 02 03 02 02 03 03
03 03 04 03 03 04 05 08 05 05 04
04 05 0a 07 07 06 08 0c 0a 0c 0c
0b 0a 0b 0b 0d 0e 12 10 0d 0e 11
0e 0b 0b 10 16 10 11 13 14 15 15
15 0c 0f 17 18 16 14 18 12 14 15
14 ff db 00 43 01 03 04 04 05 04
05 09 05 05 09 14 0d 0b 0d 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 ff c0 00 11 08 01 00
01 00 03 01 22 00 02 11 01 03 11
01 ff c4 00 1f 00 00 01 05 01 01
01 01 01 01 00 00 00 00 00 00 00
00 01 02 03 04 05 06 07 08 09 0a
0b ff c4 00 b5 10 00 02 01 03 03
02 04 03 05 05 04 04 00 00 01 7d
01 02 03 00 04 11 05 12 21 31 41
06 13 51 61 07 22 71 14 32 81 91
a1 08 23 42 b1 c1 15 52 d1 f0 24
33 62 72 82 09 0a 16 17 18 19 1a
25 26 27 28 29 2a 34 35 36 37 38
39 3a 43 44 45 46 47 48 49 4a 53
54 55 56 57 58 59 5a 63 64 65 66
67 68 69 6a 73 74 75 76 77 78 79
7a 83 84 85 86 87 88 89 8a 92 93
94 9

In [ ]:
# make huffman codes
code = ""
def get_codebook(freq):
    global code
    q = PriorityQueue()
    m = len(freq)
    n = len(freq[0])
    codebook = np.empty((m, n), dtype = object)

    cnt = 0
    for i in range(m):
        for j in range(n):
            if freq[i,j]:
                q.put((freq[i,j], str((i,j))))
                cnt += 1

    for i in range(cnt-1):
        a = q.get()
        b = q.get()
        q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

    tree = eval(q.get()[1])

    code = ""
    def create_codes(arr, codebook):
        global code
        if type(arr[1]) == int and type(arr[0]) == int:
            codebook[arr[0], arr[1]] = code
        else:
            code += "0"
            create_codes(arr[0], codebook)
            code = code[:-1]
            code += "1"
            create_codes(arr[1], codebook)
            code = code[:-1]

    create_codes(tree, codebook)
    return codebook

codes_dc_Y = get_codebook(dc_huffman_Y)
codes_ac_Y = get_codebook(ac_huffman_Y)
codes_dc_C = get_codebook(dc_huffman_C)
codes_ac_C = get_codebook(ac_huffman_C)